<a href="https://colab.research.google.com/github/bikash119/learn_tensorflow/blob/main/timeseries_mulvar_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# import statements
import tensorflow as tf
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import datetime
from pathlib import Path

In [2]:
# Download data
tf.keras.utils.get_file(fname="ridership.tgz"
                        ,origin="https://github.com/ageron/data/raw/main/ridership.tgz"
                        ,extract=True
                        ,cache_dir="."
                        )


108512/108512 [==============================] - 0s 0us/step


'./datasets/ridership.tgz'

In [3]:
pd.read_csv("/content/datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv").columns

Index(['service_date', 'day_type', 'bus', 'rail_boardings', 'total_rides'], dtype='object')

In [5]:
file_path=Path("/content/datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df= pd.read_csv(file_path,parse_dates=["service_date"])
df.columns= ["date", "day_type","bus","rail","total"]
df= df.sort_values("date").set_index("date") # Sort the data by date and set date index column
df= df.drop("total",axis=1) # remove the total column as it just the total of bus and rail rides on any given day
df.drop_duplicates()

,day_type,bus,rail
date,,,
2001-01-01,U,297192,126455
2001-01-02,W,780827,501952
2001-01-03,W,824923,536432
2001-01-04,W,870021,550011
2001-01-05,W,890426,557917
...,...,...,...
2021-11-26,W,257700,189694
2021-11-27,A,237839,187065
2021-11-28,U,184817,147830


In [17]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs,
                        callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6,history

# MultiVariate TimeSeries Single Step Forecasting

In [13]:
df_mulvar = df[["bus", "rail"]] / 1e6
df_mulvar["next_day_type"]= df["day_type"].shift(-1)
df_mulvar= pd.get_dummies(df_mulvar)
mulvar_train = df_mulvar["2016-01":"2018-12"]
mulvar_valid = df_mulvar["2019-01":"2019-05"]
mulvar_test = df_mulvar["2019-06":]

In [14]:
mulvar_train.head()

,bus,rail,next_day_type_A,next_day_type_U,next_day_type_W
date,,,,,
2016-01-01,0.303321,0.319835,1,0,0
2016-01-02,0.448859,0.365509,0,1,0
2016-01-03,0.340540,0.287661,0,0,1
2016-01-04,0.829429,0.703185,0,0,1
2016-01-05,0.846789,0.727716,0,0,1


In [15]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
seq_length= 56
train_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),  # use all 5 columns as input
    targets=mulvar_train["rail"][seq_length:],  # forecast only the rail series
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

In [16]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
mulvar_model = tf.keras.Sequential([
    # Input shape takes 5 as we are want to use all the columns for training the model
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(1)
])

In [18]:
mae,history = fit_and_evaluate(mulvar_model, train_mulvar_ds, valid_mulvar_ds,
                 learning_rate=0.05)

Epoch 1/500
33/33 [==============================] - 3s 27ms/step - loss: 0.0328 - mae: 0.1870 - val_loss: 0.0083 - val_mae: 0.1212
Epoch 2/500
33/33 [==============================] - 1s 17ms/step - loss: 0.0039 - mae: 0.0687 - val_loss: 0.0026 - val_mae: 0.0618
Epoch 3/500
33/33 [==============================] - 1s 17ms/step - loss: 0.0023 - mae: 0.0480 - val_loss: 9.0877e-04 - val_mae: 0.0311
Epoch 4/500
33/33 [==============================] - 1s 19ms/step - loss: 0.0017 - mae: 0.0419 - val_loss: 0.0010 - val_mae: 0.0345
Epoch 5/500
33/33 [==============================] - 1s 16ms/step - loss: 0.0016 - mae: 0.0403 - val_loss: 0.0014 - val_mae: 0.0432
Epoch 6/500
33/33 [==============================] - 1s 16ms/step - loss: 0.0016 - mae: 0.0397 - val_loss: 7.2439e-04 - val_mae: 0.0268
Epoch 7/500
33/33 [==============================] - 1s 18ms/step - loss: 0.0014 - mae: 0.0367 - val_loss: 8.0656e-04 - val_mae: 0.0282
Epoch 8/500
33/33 [==============================] - 1s 15ms/ste

# MultiVariate TimeSeries Single Step Multi-task Forecasting

In [19]:
tf.random.set_seed(42)

seq_length = 56
train_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),
    targets=mulvar_train[["bus", "rail"]][seq_length:],  # 2 targets per day
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid[["bus", "rail"]][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

tf.random.set_seed(42)
multask_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(2)
])

fit_and_evaluate(multask_model, train_multask_ds, valid_multask_ds,
                 learning_rate=0.02)

Epoch 1/500
33/33 [==============================] - 2s 26ms/step - loss: 0.0482 - mae: 0.2187 - val_loss: 0.0041 - val_mae: 0.0755
Epoch 2/500
33/33 [==============================] - 1s 16ms/step - loss: 0.0035 - mae: 0.0583 - val_loss: 0.0015 - val_mae: 0.0418
Epoch 3/500
33/33 [==============================] - 1s 17ms/step - loss: 0.0023 - mae: 0.0456 - val_loss: 0.0012 - val_mae: 0.0367
Epoch 4/500
33/33 [==============================] - 1s 18ms/step - loss: 0.0021 - mae: 0.0445 - val_loss: 0.0019 - val_mae: 0.0509
Epoch 5/500
33/33 [==============================] - 1s 18ms/step - loss: 0.0020 - mae: 0.0440 - val_loss: 0.0012 - val_mae: 0.0375
Epoch 6/500
33/33 [==============================] - 1s 17ms/step - loss: 0.0019 - mae: 0.0419 - val_loss: 0.0012 - val_mae: 0.0379
Epoch 7/500
33/33 [==============================] - 1s 16ms/step - loss: 0.0018 - mae: 0.0417 - val_loss: 9.9792e-04 - val_mae: 0.0338
Epoch 8/500
33/33 [==============================] - 1s 18ms/step - loss

(24589.475244283676, <keras.callbacks.History at 0x7b6e583d9960>)